In [1]:
from mindquantum.algorithm.error_mitigation import zne
from mindquantum.simulator import Simulator
from mindquantum.core.circuit import Circuit
from mindquantum.core.operators import Hamiltonian, QubitOperator

In [2]:
from mindquantum.core.gates import RX, RY, RZ
import numpy as np

def rb_circ(n_gate):
    circ = Circuit()
    for _ in range(n_gate):
        circ += np.random.choice([RX, RY, RZ])(np.random.uniform(-1, 1)).on(0)
    return circ

circ = rb_circ(50)
circ.svg()

In [3]:
from mindquantum.simulator.noise import NoiseBackend
from mindquantum.core.circuit.channel_adder import BitFlipAdder

def execute(circ: Circuit, noise_level: float, ham: Hamiltonian):
    return Simulator(NoiseBackend('mqmatrix', circ.n_qubits, BitFlipAdder(noise_level))).get_expectation(ham, circ)

In [6]:
ham = Hamiltonian(QubitOperator('Z0'))
noise_level = 0.001
true_value = execute(circ, 0.0, ham)
noisy_value = execute(circ, noise_level, ham)
zne_value = zne(circ, execute, scaling=np.linspace(1, 3, 3), args=(noise_level, ham))

In [7]:
print(f"true_value: {true_value}")
print(f"noisy_value: {noisy_value}")
print(f"zne_value: {zne_value}")
print(f"Error (w/o  Mitiq): %0.4f" % abs((true_value - noisy_value) / true_value))
print(f"Error (with Mitiq): %0.4f" % abs((true_value - zne_value) / true_value))

true_value: (-0.70121757539835+0j)
noisy_value: (-0.6472216034269564+0j)
zne_value: (-0.6917878583940983+0j)
Error (w/o  Mitiq): 0.0770
Error (with Mitiq): 0.0134
